In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05866122245788574
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 6.502140042995631
200000 10.751036315579071
300000 11.703452849398598
400000 11.79046195840332
500000 11.426640468970398
600000 13.817740368221223
700000 14.432255942541838
800000 16.743439846060813
900000 19.2128988767438
1000000 20.710749716373243
1100000 18.36176645257982
1200000 9.52868347704213
1300000 20.59665894214045
1400000 20.826843002821878
1500000 20.752948719505852
1600000 21.134295430850955
1700000 21.89902460148868
1800000 21.523630879059684
1900000 21.013821070134384
2000000 21.328286394415606
2100000 21.62861431053008
2200000 21.38886341508293
2300000 5.689374273372641
2400000 24.01420844919746
2500000 27.288210853586286
2600000 26.753289375430207
2700000 26.524761110848594
2800000 27.22001910179323
2900000 26.173438039419437
3000000 26.090

29500000 7.7047427170659
29600000 23.344206854972043
29700000 22.230721089619436
29800000 21.446760630788305
29900000 22.109489869687987
30000000 20.79930960219635
30100000 21.829367715429953
30200000 20.384719054365547
30300000 20.49561196747129
30400000 5.888726681642741
30500000 20.273939126332994
30600000 20.785009117691327
30700000 20.579785853751552
30800000 21.858214203291052
30900000 20.974881192437174
31000000 21.478905406542925
31100000 22.171088965026932
31200000 21.283540741684217
31300000 8.95500030099061
31400000 21.48717545942378
31500000 21.911527530813263
31600000 21.765338967356087
31700000 18.26709878734399
31800000 11.233844867644834
31900000 20.502749968422016
32000000 22.176956730484928
32100000 20.81725075612426
32200000 20.190486888643473
32300000 19.79357807855803
32400000 19.274282667342405
32500000 19.103770223232985
32600000 12.01106683896255
32700000 13.585331923246398
32800000 18.747961324141492
32900000 19.225642344717112
33000000 19.434502322556135
33100

59200000 17.74071253074494
59300000 17.323928514113685
59400000 17.331056024424985
59500000 16.97790935060361
59600000 7.586496600520408
59700000 15.246180730174922
59800000 17.5564859066343
59900000 17.1485377133563
60000000 17.049751361531072
60100000 18.85650023667106
60200000 18.05432280727871
60300000 7.619060125385642
60400000 18.738964112467826
60500000 19.748549723873708
60600000 19.444347855521986
60700000 18.773462152118046
60800000 20.204320959108962
60900000 18.327819920040973
61000000 19.40048654821819
61100000 18.62060196996272
61200000 8.944186041107493
61300000 17.598071036345086
61400000 17.42659143983822
61500000 17.82979766703774
61600000 16.67439563280921
61700000 17.096810290797166
61800000 13.923913445239812
61900000 8.696367149728864
62000000 17.343784769027554
62100000 15.953554829652258
62200000 17.08023472260846
62300000 16.918056529985435
62400000 17.62869206919803
62500000 9.376655268128076
62600000 15.57528240963436
62700000 19.202923008578342
62800000 18.3

89000000 15.051762103366235
89100000 14.795112474468853
89200000 14.823205426653109
89300000 6.486268901157031
89400000 15.425454335061302
89500000 15.326774035544304
89600000 15.906658248851786
89700000 16.071396852611155
89800000 16.129203871362478
89900000 16.064747141925636
90000000 16.57618407903319
90100000 15.87945360629651
90200000 16.57631255593677
90300000 15.740087144964312
90400000 15.736169635498353
90500000 14.821262610926572
90600000 12.286451952783004
90700000 8.577778733601571
90800000 14.19383004926918
90900000 14.112662172123862
91000000 14.60378199948731
91100000 4.2029391580004
91200000 14.886168845046837
91300000 14.815855880445431
91400000 14.27248719080479
91500000 8.125617745653825
91600000 13.268696514860988
91700000 14.427919717302535
91800000 15.386141055261332
91900000 15.545740027337791
92000000 15.59693940022104
92100000 15.755535185394681
92200000 16.286048446048486
92300000 15.897637328389171
92400000 16.183049650015885
92500000 15.78251286590147
926000

118000000 10.694300241215636
118100000 5.730970054890971
118200000 11.947061531849945
118300000 5.346447612910569
118400000 12.804647939250204
118500000 12.592205261680773
118600000 12.416857572212757
118700000 12.633109460105308
118800000 12.68265764221797
118900000 12.718647511582892
119000000 13.656134286336584
119100000 14.076143826523321
119200000 13.890054927571386
119300000 14.153616517693406
119400000 13.179542093968758
119500000 12.881524938833147
119600000 12.514988890343371
119700000 12.971210648238243
119800000 12.418994197627157
119900000 12.430788863109555
120000000 12.561877212350403
120100000 5.663196443081876
120200000 12.357189766777006
120300000 5.112952719434424
120400000 11.00057176370105
120500000 6.69826548172664
120600000 10.759446617820638
120700000 12.895445277553328
120800000 12.207025334977876
120900000 12.818864863387418
121000000 12.55189695335115
121100000 12.52101423344898
121200000 13.263285321222968
121300000 13.358146028035195
121400000 13.96363462744

...

175600000 9.062836123087148
175700000 8.162900350811418
175800000 8.441079157286525
175900000 8.121151894732556
176000000 8.290167317896788
176100000 8.362012681786593
176200000 8.090255472687215
176300000 3.554085154666649
176400000 7.835332119284249
176500000 3.5261895285448532
176600000 7.164725687360544
176700000 5.568648484216206
176800000 6.087916576422762
176900000 8.131336658686227
177000000 8.305769139334242
177100000 8.035154184873909
177200000 8.286879053268988
177300000 8.44573837985366
177400000 8.424969990146826
177500000 9.31764057742102
177600000 8.888644216853141
177700000 9.488734929484217
177800000 9.035432171757417
177900000 9.035989228571571
178000000 8.497337726916792
178100000 8.432030491624733
178200000 8.587635184010475
178300000 8.065258803557464
178400000 8.408030253117253
178500000 5.045720167195702
178600000 7.195149874439984
178700000 6.800334128763672
178800000 3.982762206135253
178900000 8.280294790459216
179000000 3.5796207924505206
179100000 8.14677102

205000000 7.214702960838318
205100000 6.722142606558068
205200000 6.914345160748125
205300000 2.8223049447034794
205400000 6.7779533520999875
205500000 6.698267999972414
205600000 6.529533742420366
205700000 4.518503011726476
205800000 4.519669559737799
205900000 6.637195975173235
206000000 6.9986910183607005
206100000 6.761842450880902
206200000 2.4545027720813652
206300000 6.741008211141126
206400000 6.997420821971821
206500000 7.318516428342353
206600000 7.250014915446526
206700000 7.549374368175755
206800000 7.27047569842571
206900000 7.249484425566017
207000000 7.190648990653312
207100000 6.967239124334375
207200000 6.849181203675455
207300000 6.936756127543929
207400000 6.61810196832052
207500000 3.9514877557182384
207600000 5.361232079012451
207700000 6.466241988120406
207800000 6.208376307220048
207900000 6.745741526435938
208000000 1.8157508200625478
208100000 6.489167191966117
208200000 6.380875245293346
208300000 6.266520345817546
208400000 5.267710687557528
208500000 3.9874

234200000 5.273768188847515
234300000 1.9130820999092757
234400000 5.1853837626701
234500000 4.7118819306964745
234600000 4.557002499090805
234700000 4.7058302874698645
234800000 4.4169615851074955
234900000 4.2071488100354575
235000000 2.0265423544571
235100000 4.340145282601373
235200000 4.772474777035849
235300000 4.412585400446365
235400000 4.889321168903834
235500000 4.875938298946858
235600000 3.407575638212621
235700000 3.552497709298836
235800000 5.007242305081602
235900000 5.343292313386848
236000000 4.989953039019429
236100000 5.104509744620211
236200000 4.902308421449175
236300000 5.193785778866555
236400000 5.025468575234596
236500000 3.112254567430526
236600000 3.7541278735433288
236700000 4.964096635720131
236800000 4.3272824825904666
236900000 4.602301841189368
237000000 4.525402272547745
237100000 4.193595506309639
237200000 2.1732349726054867
237300000 3.8352754501283437
237400000 4.316431519796848
237500000 4.634515403697706
237600000 4.616772515122039
237700000 4.937

263100000 2.756323539001958
263200000 2.5444472817124826
263300000 1.0589236493768064
263400000 2.6010752901421057
263500000 2.646455757919252
263600000 2.5278115257149274
263700000 2.554479824614601
263800000 2.532887698407491
263900000 2.3952198982607427
264000000 2.4860882886039914
264100000 2.2822741220167675
264200000 0.6963283480178879
264300000 2.4890973559530774
264400000 2.297766565554589
264500000 2.5601024565862684
264600000 2.393398641680542
264700000 2.591017295376519
264800000 2.4809097342100705
264900000 2.524069292761322
265000000 2.722373922214826
265100000 0.9040380508264271
265200000 2.6263585876748397
265300000 2.4087351222557425
265400000 2.490713053140723
265500000 1.5010705333832073
265600000 1.8456672192929486
265700000 2.4258350900618204
265800000 2.306775642636905
265900000 2.3003462769900263
266000000 2.2953429525063336
266100000 2.2341621150787043
266200000 2.2409138541365268
266300000 2.0344319968080122
266400000 1.4117597604071597
266500000 1.5219241229974

291600000 0.43045745661673945
291700000 0.4104097982113361
291800000 0.4380799384437866
291900000 0.39568083660964504
292000000 0.4052542102504969
292100000 0.36345089152557986
292200000 0.32473243443211247
292300000 0.07971984026578069
292400000 0.26715943349945215
292500000 0.2759280626137058
292600000 0.2748338774286409
292700000 0.2657877848317391
292800000 0.25857335831678907
292900000 0.2555859239174525
293000000 0.2262241542463799
293100000 0.23551273022965258
293200000 0.2274401119966507
293300000 0.19569366377679176
293400000 0.09653177097459965
293500000 0.1766666899300118
293600000 0.1734441629368994
293700000 0.184245558859547
293800000 0.16326936893897256
293900000 0.17636184347427886
294000000 0.17750016166161167
294100000 0.13393533011358977
294200000 0.1708080888604058
294300000 0.15503818348318668
294400000 0.14715590538448095
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.06455023159726792
294600000 0.011639529320418835
294700000 0.